In [4]:
import pandas as pd
import numpy as np
import sys
import os
from matplotlib import pyplot as plt

from gensim.models import Word2Vec

In [20]:
def get_Xy(df, model, max_word_n):

    def tune_size(i, words):
        words = [model.wv[word] for word in words][:max_word_n]
        if i%100 == 0:
            percent = i/data_n*100
            sys.stdout.write("\r% 5.2f%%"%(percent))
        return np.array(words+placeholder[len(words):])

    data_n = len(df)
    placeholder = [np.array([0]*100)]*max_word_n
    
    X = np.array([tune_size(i, words)
                  for i, words in enumerate(df["review"])])
    sys.stdout.write("\r% 5.2f%%"%100)
    
    y = np.array([[1, 0] if l == "neg" else [0, 1] for l in df["label"]])
    
    return X, y

In [21]:
cases = ["normal", "underscore"]
case = "normal"

if case == "normal":
    max_word_n = 80
elif case == "underscore":
    max_word_n = 160

df = pd.read_pickle("./data/%s_df"%case)
model = pd.read_pickle("./data/%s_model"%case)

X, y = get_Xy(df, model, max_word_n)

 99.80%

ValueError: could not broadcast input array from shape (80,128) into shape (80)

28981

In [111]:
lengths = [len(words.split()) for words in df["review"]]
df = df.iloc[[False if l > max_length else True for l in lengths]]
data_n = len(df)
print(data_n)

sent_se = pd.Series([words.split() for words in df["review"]])

plt.figure(figsize=[15, 5])
plt.hist(lengths, bins=300)
plt.show()

def save_batch_csv(df , type_, n):
    df = df[df["type"] == type_]
    pos_df = df[df["label"] == "pos"]
    neg_df = df[df["label"] == "neg"]
    df = pd.concat([pos_df[:n//2], neg_df[:n//2]])
    df.to_csv("./data/batch_of_%s.csv"%type_)

save_batch_csv(df, "test", 2000)
save_batch_csv(df, "train", 8000)